In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.consequents/submit_results.csv /content/consequent_results.csv

In [0]:
import pandas as pd

prefix = '/content/'

test_df = pd.read_csv(prefix + 'train.csv', header=None) # change to test.csv
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], # comment out following for test.csv
    'antecedent': test_df[2],
    'consequent': test_df[3],
    'astart':test_df[4],
    'aend':test_df[5],
    'cstart':test_df[6],
    'cend':test_df[7]
})

# display 
test_df.head()

In [0]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2],
    'start':a_df[3],
    'end': a_df[4]
})
a_df.sort_values(by=['id','text'], ascending=[1,0])

c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2],
    'start':c_df[3],
    'end': c_df[4]
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})

a_df.head(30)

In [0]:
pip install allennlp==1.0.0rc4 allennlp-models==1.0.0rc4

In [0]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.syntax.biaffine_dependency_parser
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

original = "When they tested a strategy that sold stocks and went into cash every time price-to-dividend multiples went clearly above their historic mean at the time, and re-entered when they had become cheap, in 20 different countries, they found that in all of them it would have been better simply to buy and hold stocks."    
prediction = predictor.predict(
  sentence = original
)
prediction

In [0]:
def extract_dependents(prediction):
 root = None
 text = str()

 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for key, val in v.items():
      if key=='root':
        root = val
      if key=='text': 
        text = val   

 root.update({'text': text})
 return extract_children(root) 

In [0]:
def extract_children(obj, include='', exclude=''):
  arr = list()
  deps = list()
  spans = list()
  verbs = list()

  def extract(obj, arr): 
    explore = False
    limit = 5 
    count = 0
    text = str()

    if isinstance(obj, dict):
      for k, v in obj.items():
        if include =='' and k=='attributes' and ('VERB' in v or 'AUX' in v or 'PROPN' in v or 'NOUN' in v or 'PREP' in v):
          if obj not in verbs:
            verbs.append(obj)

        if k=='nodeType':
          arr.append(v)

        if k=='text' and len(v) > 0 and len(text) == 0:  
          text = v

        if isinstance(v, (dict, list)):
          if count < limit:
            count = count + 1
            extract(v, arr) # children   
             
    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
    
          for key, value in item.items():
            if key == 'nodeType':
              if (include=='' or value in include) and (exclude=='' or not [v for v in arr if v in exclude]):
                #print(include, exclude, item)
                if item not in deps: 
                  deps.append(item) # child
                explore = True 
              else:  
                explore = False

            if include=='' and key=='attributes' and ('VERB' in value or 'AUX' in value or 'PROPN' in value or 'NOUN' in value or 'PREP' in value):
              explore = True 
            elif key=='attributes' and 'CCONJ' in value:
              explore = False     

            if explore and key =='spans':
              if value not in spans:
                spans.append(value)  
            
          if explore and count < limit:
            count = count + 1
            extract(item, arr) 
         
    return text

  text = extract(obj, arr)
  return text, spans, deps, verbs

In [0]:
def position(span):
  return span[0]['start']

In [0]:
def follow_children(obj):
  arr = list()

  def follow(obj, arr): 

    if isinstance(obj, dict):
      for k, v in obj.items():
        if k == 'children':      
          follow(v, arr)

    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
          for key, value in item.items():
            if key =='attributes' and ('VERB' in value or 'AUX' in value or 'PROPN' in value or 'NOUN' in value or 'PREP' in value):
              arr.append(item)

          follow(item, arr)
      
    return arr

  results = follow(obj, arr)
  return results

In [0]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [0]:
def normalize(fragment):
  r = fragment.strip()
  r = compact(r)
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [0]:
def explore(verb, condition, complement, sentence, include='',exclude=''):
    deps = list()
    loops= list()
    if len(condition)==0:
      return deps, loops

    begin = sentence.find(condition) # superspan for condition
    if begin ==-1: 
      #print(condition,' not in:\n'+sentence)
      return deps, loops
    stop = begin + 1 + len(condition) 

    if len(complement) > 0:
      index = sentence.find(complement) # superspan for complement
      if index ==-1: 
        #print(complement,' not in:\n'+sentence)
        return deps,loops
      if begin < index and index < stop: 
        #print(complement,'<-->'+condition,' in:\n'+sentence)
        loops.append(complement)
        deps.append(condition)   
      elif index == stop:  
        #print(complement,' + '+condition)
        stop = stop + 1 + len(complement)
        loops.append(complement)
      #elif index > stop:
      #  stop = index 
      #  condition = sentence[begin:stop]

    word = verb['word']
    #print( word ,'-->', condition)
    subtext, subspans, subjs, subverbs = extract_children(verb, include, exclude)
    if len(subspans) == 0:
      return deps, loops

    for sub in subjs:    
      descendant = sub['word']
      #print(word, begin, '\"'+descendant+'\"', stop) 
      for span in sorted(subspans,key=position): # all subspans
        start=span[0]['start']
        end = span[0]['end']
        #print(start, sentence[start:end], begin, stop)
        if begin <= start and start <= stop: # span of the condition
          if start > 2: start = start - 2 # due to some unknown issue
          spantext = sentence[start:end].strip()
          #print(spantext, start, condition, end)
          if len(spantext) > 0:
            if spantext in condition:
              if spantext not in deps:
                deps.append(spantext)   
            if spantext in complement:
              if spantext not in loops: 
                loops.append(spantext)  

    return deps, loops       

In [0]:
def track(verbs, antecedent, consequent, sentence):
  adeps = set()
  aloops= set()
  cdeps = set()
  cloops= set()
  if len(verbs)==0:
    print('no verbs!')
    return adeps, cdeps, aloops, cloops

  for verb in verbs: # any 'VERB' or "AUX" (Why ?)
    deps, loops = explore(verb, antecedent, consequent, sentence, 'dep nsubj','cc advcl')
    adeps.update(deps)
    aloops.update(loops)
    deps, loops = explore(verb, consequent, antecedent, sentence, 'dep prep advmod aux','xcomp')
    cdeps.update(deps)
    cloops.update(loops)
    if len(adeps) > 0 and len(cdeps) > 0:
      return adeps, cdeps, aloops, cloops    
    adeps = set()
    cdeps = set()  

  return adeps, cdeps, aloops, cloops   

In [0]:
def separate_from(md, pa):
  index = -1
  if pa in md and len(md) > len(pa):
    index = md.find(pa)
    if index > 0 :
      md = md[0:index]
      #print('|'+md+'{...}')
    elif index == 0:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md, index       

In [0]:
def analyze(id, df, r_df, sentence, verbs=[], ctext='', first=True):
  atext = str()
  astart = -1
  aend = -1
  arule = df.loc[df.id.str.contains(id)]
  if len(arule) == 0:
    return id, 0, atext, astart, aend, 0

  print('|',ctext)
  # First check based on rules
  #print(id,len(arule.index))
  alab ='0' # assume antecedent false
  count= 0  # counter of arules
  for a in arule.index: # [a] = index
    count = count + 1
    amid = arule['id'][a]  
    alab = arule['labels'][a] # rule
    astart = arule['start'][a]
    aend = arule['end'][a]
    atext = arule['text'][a]
    text = atext
    #print(amid,'candidate ='+atext)
    if alab == 1:
      print('v' , atext)
      return amid, alab, atext, astart, aend, 100 # positive!

    # If rules gave no certain "yes"(==1 above)
    apred = r_df.loc[r_df.id.str.contains(amid)]
    if len(apred) == 0:
      print(amid, ' not predicted')
      continue
    for p in apred.index: # [p] = index
      amid = apred['id'][p]  
      alab = apred['labels'][p] # prediction
      apredrule = df.loc[df.id.str.contains(amid)]
      if len(apredrule) == 0:
        print(amid, ' not found ')
        continue
      for i in apredrule.index:    
        astart = apredrule['start'][i]
        aend = apredrule['end'][i]
        atext = apredrule['text'][i]
        text = atext
        #print(a,amid,alab,atext)

        print('|', atext) 
        if alab == 1: # positive prediction (what if there are multiple?) 
          print('v')
          return amid, alab, atext, astart, aend, 75 # certain but not 100 
        else:
          if len(ctext) > 0:
            if atext == ctext or ctext in atext or atext in ctext:
              if len(arule.index) > count:
                print('x',count)
                continue  
              atext, index = separate_from(atext, ctext)# restore
              print('$',atext)
              return amid, alab, atext, astart, aend, 0 # 0=Error

            adeps,cdeps,aloops,cloops = track(verbs, atext, ctext, sentence)
            print(adeps,cdeps,aloops,cloops)

            if len(adeps)==0 and len(cdeps)==0 and len(aloops)==0 and len(cloops)==0:
              if first and len(arule.index) > count:
                print('x',count)
                continue 
              return amid, alab, atext, astart, aend, '-25' # very unlikely

            if len(adeps)>0 and len(cdeps)>0 and len(aloops)>0 and len(cloops)>0:
              if len(arule.index) > count:
                print('x',count)
                continue    
              return amid, alab, atext, astart, aend, '-75' # should be impossible

            if len(adeps) > 0 and len(cdeps) > 0: # relaxed for correction
              if first and len(arule.index) > count:
                print('x',count)
                continue
            if len(aloops)>0 or len(cloops) >0:
              if len(aloops)>0:
                atext = atext + ' ' + ctext
              else:
                atext = ctext + ' ' + atext
              aend = aend + 1 + len(ctext) # plus 1 space 
              print('+' , atext)   
              return amid, alab, atext, astart, aend,'-50' # certainly-not
            print('v')              
            return amid, alab, atext, astart, aend, 50 # uncertain     
  print('v')
  return amid, alab, atext, astart, aend, 25 # unpredicted

In [0]:
original = "If that was my daughter, I would have asked If I did something wrong."    

prediction = predictor.predict(sentence = original)
sentence, spans, deps, verbs = extract_dependents(prediction)

verbs = follow_children(verbs) # flatten the tree
antecedent = "If that was my daughter"
consequent = "would have asked If I did something wrong"

sentence = normalize(sentence)
adeps, cdeps, aloops, cloops = track(verbs, antecedent, consequent, sentence)
print(adeps,cdeps)
print(aloops,cloops)

In [0]:
import pandas as pd

prefix = '/content/'

from tqdm import tqdm
from time import sleep

submit_df = pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])
report_df = pd.DataFrame(columns=['sentenceID','clabel','cprob','cdep','consequent','cstart','cend','gold_consequent','gold_antecedent','antecedent', 'alabel','aprob','astart','aend'])

count = 0
for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]
  print(id)

  sentence = original.replace('\"','')
  prediction = predictor.predict(sentence)
  rebuilt, spans, deps, verbs = extract_dependents(prediction)
  verbs = follow_children(verbs) # flatten the   
  sentence = normalize(rebuilt)

  astart=-1
  aend = -1
  cstart=-1
  cend = -1
  text =str()
  atext=str()  
  ctext=str()

  cid, clab, ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext)
  aid, alab, atext, astart, aend, aprob = analyze(id, a_df, ra_df, sentence, verbs, ctext)
  if aprob == '-50':
    ctext = atext
    aid, alab, atext, astart, aend, aprob = analyze(id, a_df, ra_df, sentence, verbs, ctext, False)
    if aprob == 0:
      cid,clab,ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext, False)  
  else:  
    cid, clab, ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext, False)
    if cprob == '-50':
      atext = ctext
      cid, clab, ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext, False)

  consequent = ctext
  antecedent = atext
  print(antecedent)
  print(consequent)

  cdep = '' # Pass
  if consequent == antecedent:
    consequent = ''
    cstart=-1
    cend = -1
    cdep = '='
  if len(verbs) > 0 and len(consequent) > 0 and not consequent in antecedent:
    adeps,cdeps,aloops,cloops = track(verbs, antecedent, consequent, sentence)

    if len(adeps) > 0 and len(cdeps) > 0 and len(aloops) > 0 and len(cloops)>0:
      antecedent = consequent
      astart=cstart
      aend = cend
      aprob= cprob
      consequent = ''
      cstart=-1
      cend = -1
      cdep = '>'
    elif len(adeps)>0 and len(cdeps)>0:
      cdep = '?'  

  if len(antecedent) > 0:       
    if len(consequent) > 0:
      text = antecedent + ' | ' + consequent
    else:
      text = antecedent
  else:
    cdep = '!'      

  print(cdep) # post-correction
  if len(cdep) > 0:
    count=count+1

  report_row = pd.Series( data={
    'sentenceID': id,
    'clabel': clab,
    'cprob': cprob,
    'cdep' : cdep,
    'consequent': consequent, # comment out gold_,
    'cstart': cstart,
    'cend': cend,
    'gold_consequent': test_df['consequent'][ind],
    'gold_antecedent': test_df['antecedent'][ind],
    'antecedent': antecedent, 
    'alabel': alab,
    'aprob': aprob,
    'astart': astart,
    'aend': aend
  }, name = str(id) )

  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  }, name = str(id) )

  submit_df = submit_df.append(result_row, ignore_index=True)
  report_df = report_df.append(report_row, ignore_index=True)

  print(id, text, astart, aend, cstart, cend)
  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid']) 
report_df.to_csv(prefix+'report.csv',index=False, header=['sentenceID','clabel','cprob','cdep','consequent','cstart','cend','gold_consequent','gold_antecedent','antecedent','alabel','aprob','astart','aend'])
print('Falied =',count)

In [0]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/report.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/